In [64]:
!pip install yfinance pandas numpy scikit-learn torch



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [65]:
import yfinance as yf
import pandas as pd
import numpy as np
import torch
from torch import nn
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt



In [66]:
# Download 5 years of AAPL daily data
start_date = "2019-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

df = yf.download("AAPL", start=start_date, end=end_date, interval="1d")
df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
df.dropna(inplace=True)

# Add engineered features
df['Return'] = df['Close'].pct_change()
df['Candle_Body'] = df['Close'] - df['Open']
df['Range'] = df['High'] - df['Low']
df.dropna(inplace=True)


/var/folders/l6/kt268nls2sqcx51m4w_z09pc0000gn/T/ipykernel_59995/3827814912.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download("AAPL", start=start_date, end=end_date, interval="1d")
[*********************100%***********************]  1 of 1 completed


In [67]:
# Select and scale features
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'Return', 'Candle_Body', 'Range']
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[features])

# Create sequences
SEQ_LEN = 60
X, y = [], []

for i in range(SEQ_LEN, len(scaled_data)):
    window = scaled_data[i-SEQ_LEN:i]
    today_close = scaled_data[i][features.index('Close')]
    prev_close = scaled_data[i-1][features.index('Close')]
    change = today_close - prev_close
    threshold = 0.002

    if change > threshold:
        X.append(window)
        y.append(1)  # up
    elif change < -threshold:
        X.append(window)
        y.append(0)  # down
    else:
        continue  # skip small movements

X = np.array(X)
y = np.array(y)


In [68]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")


X_train shape: torch.Size([1003, 60, 8])
y_train shape: torch.Size([1003])


In [69]:
import torch
import torch.nn as nn

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size, 1)

    def forward(self, lstm_output):
        # lstm_output: (batch, seq_len, hidden_size)
        weights = self.attn(lstm_output)               # [batch, seq_len, 1]
        weights = torch.softmax(weights.squeeze(-1), dim=1)  # [batch, seq_len]
        context = torch.bmm(weights.unsqueeze(1), lstm_output).squeeze(1)  # [batch, hidden_size]
        return context, weights

class StockLSTMStable(nn.Module):
    def __init__(self, input_size=8, hidden_size=128, num_layers=2, dropout_rate=0.4):
        super(StockLSTMStable, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_rate
        )
        self.attention = Attention(hidden_size)
        self.bn = nn.BatchNorm1d(hidden_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 2)  # Output: up/down
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        attn_out, _ = self.attention(lstm_out)
        out = self.bn(attn_out)
        out = self.dropout(out)
        return self.fc(out)


In [70]:
model = StockLSTMStable(input_size=8, hidden_size=128, num_layers=2, dropout_rate=0.4)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 20  # More cycles, better generalization

for epoch in range(EPOCHS):
    model.train()
    output = model(X_train)
    loss = loss_fn(output, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    preds = output.argmax(dim=1)
    acc = (preds == y_train).float().mean()
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {loss.item():.4f} | Train Accuracy: {acc.item():.4f}")


Epoch 1/20 | Loss: 0.7168 | Train Accuracy: 0.4855
Epoch 2/20 | Loss: 0.7104 | Train Accuracy: 0.5304
Epoch 3/20 | Loss: 0.7039 | Train Accuracy: 0.5404
Epoch 4/20 | Loss: 0.7022 | Train Accuracy: 0.5155
Epoch 5/20 | Loss: 0.6929 | Train Accuracy: 0.5155
Epoch 6/20 | Loss: 0.6942 | Train Accuracy: 0.5194
Epoch 7/20 | Loss: 0.6989 | Train Accuracy: 0.5194
Epoch 8/20 | Loss: 0.6942 | Train Accuracy: 0.5214
Epoch 9/20 | Loss: 0.6913 | Train Accuracy: 0.5304
Epoch 10/20 | Loss: 0.6982 | Train Accuracy: 0.5085
Epoch 11/20 | Loss: 0.6988 | Train Accuracy: 0.5125
Epoch 12/20 | Loss: 0.6867 | Train Accuracy: 0.5324
Epoch 13/20 | Loss: 0.7048 | Train Accuracy: 0.5145
Epoch 14/20 | Loss: 0.7021 | Train Accuracy: 0.5035
Epoch 15/20 | Loss: 0.6965 | Train Accuracy: 0.5015
Epoch 16/20 | Loss: 0.6888 | Train Accuracy: 0.5274
Epoch 17/20 | Loss: 0.6977 | Train Accuracy: 0.5284
Epoch 18/20 | Loss: 0.6993 | Train Accuracy: 0.5204
Epoch 19/20 | Loss: 0.6883 | Train Accuracy: 0.5394
Epoch 20/20 | Loss: 0

In [71]:
model.eval()
with torch.no_grad():
    output_test = model(X_test)
    pred_test = output_test.argmax(dim=1)
    test_acc = (pred_test == y_test).float().mean()

print(f"Test Accuracy: {test_acc.item():.4f}")


Test Accuracy: 0.4143
